In [1]:
import pandas as pd
import math
import numpy as np
import json

from pathlib import Path
import scipy.stats as st

In [2]:
# Collect predictions

def check_num_epochs(run):
    if Path(run / "val_log.csv").is_file():
        with open(run / 'config.json') as f:
            cfg = json.load(f)

        num_epochs = cfg['num_epochs']
        val_iter = cfg['validate_iter']
        num_epochs = int(num_epochs / val_iter)

        valid_log = pd.read_csv(run / "val_log.csv", header=None, index_col=0)
        if (len(valid_log.index)-1) < num_epochs:
            print("Wrong number of epochs in run: {}".format(run))
    else:
        print("val_log.csv not exists in run: {}".format(run))

def check_only_one_tensorboard(run):
    if len(list(Path(run / "runs").glob('*'))) > 1:
        print("More than 1 tensorboard folder in run: {}".format(run))

def collect_one(model_name, run, csv_file):
    check_num_epochs(run)
    check_only_one_tensorboard(run)
    
    with open(run / 'config.json') as f:
        cfg = json.load(f)

    run_number = cfg['seed']
    regime, inv_temp = float(run.parent.parts[-1].rsplit('-', 1)[1]), float(run.parent.parts[-2].rsplit('-', 1)[1])

    csv_path = run / csv_file
    if not csv_path.exists():
        print(f'Skipping not found: {csv_path}')
        return pd.DataFrame()
    
    data = pd.read_csv(csv_path)
    if data.empty:
        print(f'Pred file is empty: {csv_path}')

    data['model'] = model_name
    data['run_number'] = int(run_number)
    data['inv_temp'] = int(inv_temp)
    data['regime'] = int(regime)
    
    return data

def collect_all(model_name, root, csv_file, regimes=['1', '2', '5', '10', '20','100'], ignore_outliers=True):
    root = Path(root)

    metrics = []
    for inv_temp in list(root.glob("inv_temp-*")):
        for regime in list(inv_temp.glob("regime-*")):
            if regime.name.rsplit("-", 1)[1] in regimes:
                for run in list(regime.glob("run-*")):
                    if ignore_outliers and 'outlier' in run.as_posix():
                        continue
                    else:
                        metrics.append(collect_one(model_name, run, csv_file))
        
    metrics = pd.concat(metrics, ignore_index=True)
    
    return metrics

In [3]:
# Compute metrics for each detected run

def compute_metrics(data, grouping, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance']):
    metrics_dict_names = {
        'Dice': 'dice',
        'Jaccard': 'jaccard',
        'Hausdorff Distance': '95hd',
        'Average Surface Distance': 'asd'
    }

    columns = ['Model', '# Run', 'Inv Temp', 'Regime']
    columns.extend(metric_names)
    metrics = []
    
    data = data.copy().reset_index()
    grouped = data.groupby(grouping)
    
    for model_group, predictions in grouped:
        model_name, run_number, inv_temp, regime = model_group[0], int(model_group[1]), int(model_group[2]), int(model_group[3])
        
        metric_values = []
        for metric_name in metric_names:
            values = predictions['segm/{}'.format(metrics_dict_names[metric_name])].values
            mean_value = np.nanmean(values)
            if metric_name == 'Dice' or metric_name == 'Jaccard':
                metric_values.append((math.ceil(mean_value*10000)/10000)*100)
            else:
                if not np.isnan(mean_value):
                    metric_values.append(math.ceil(mean_value*100)/100)
                else:
                    metric_values.append(mean_value)
        
        metrics.append([model_name, run_number, inv_temp, regime, *metric_values])
        
    metrics_df = pd.DataFrame(metrics, columns=columns)
    
    return metrics_df

def summarize_metrics(metrics, metric_names=['Dice', 'Jaccard', 'Hausdorff Distance', 'Average Surface Distance'], confidence_level=0.90, return_ranges=False):

    def compute_ci(values, return_ranges=False):
        ci = st.t.interval(confidence_level, len(values)-1, loc=np.mean(values), scale=np.std(values)/(len(values)**0.5))

        if return_ranges:
            return ci
        else:
            return (ci[1]-ci[0]) / 2

    #mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', np.mean), ('STD', np.std), ("CI {}%".format(confidence_level), compute_ci)])
    mean_metrics = metrics.groupby(['Model', 'Inv Temp', 'Regime'])[metric_names].aggregate([('Mean', 'mean'), ("CI {}%".format(confidence_level), compute_ci)]) 

    return mean_metrics

<h1>GlaS Dataset</h1>

<h2>Evaluation - Searching temperature hyperparameter</h2>

<p>Evaluate Hebbian models belonging to SWTA paradigm to search best temperature values (this value is dataset-specific)</p>

In [22]:
EXP_ROOT = "./runs"

REGIMES = ['100']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/hebbian_unsup/').glob('unet_swta_t')),
}

In [23]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [24]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'])

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,100,42.42,26.92
1,H-UNet-SWTA-T,0,5,100,46.53,30.32
2,H-UNet-SWTA-T,0,10,100,43.84,28.07
3,H-UNet-SWTA-T,0,20,100,49.06,32.50
4,H-UNet-SWTA-T,0,50,100,48.60,32.10
5,H-UNet-SWTA-T,0,75,100,46.46,30.26
6,H-UNet-SWTA-T,0,100,100,46.01,29.88


/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2244: RuntimeWarning: invalid value encountered in multiply
  lower_bound = _a * scale + loc
/home/luca/anaconda3/envs/xnet/lib/python3.10/site-packages/scipy/stats/_distn_infrastructure.py:2245: RuntimeWarning: invalid value encountered in multiply
  upper_bound = _b * scale + loc


Dice         Jaccard        
                                Mean CI 0.9%    Mean CI 0.9%
Model         Inv Temp Regime                               
H-UNet-SWTA-T 1        100     42.42     NaN   26.92     NaN
              5        100     46.53     NaN   30.32     NaN
              10       100     43.84     NaN   28.07     NaN
              20       100     49.06     NaN   32.50     NaN
              50       100     48.60     NaN   32.10     NaN
              75       100     46.46     NaN   30.26     NaN
              100      100     46.01     NaN   29.88     NaN

In [31]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    'H-UNet-SWTA-T': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('h_unet_swta_t')),
}

In [32]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

val_log.csv not exists in run: runs/GlaS/semi_sup/h_unet_swta_t/inv_temp-75/regime-10/run-0
Skipping not found: runs/GlaS/semi_sup/h_unet_swta_t/inv_temp-75/regime-10/run-0/test.csv


In [33]:
# Computing metrics
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,H-UNet-SWTA-T,0,1,1,67.77,51.25
1,H-UNet-SWTA-T,0,1,2,67.83,51.32
2,H-UNet-SWTA-T,0,1,5,67.81,51.29
3,H-UNet-SWTA-T,0,1,10,69.21,52.91
4,H-UNet-SWTA-T,0,1,20,69.68,53.46
...,...,...,...,...,...,...
325,H-UNet-SWTA-T,9,100,1,67.77,51.25
326,H-UNet-SWTA-T,9,100,2,67.88,51.38
327,H-UNet-SWTA-T,9,100,5,77.73,63.56
328,H-UNet-SWTA-T,9,100,10,79.19,65.55


Dice           Jaccard          
                                 Mean   CI 0.9%    Mean   CI 0.9%
Model         Inv Temp Regime                                    
H-UNet-SWTA-T 1        1       68.013  0.126446  51.530  0.145660
                       2       68.370  0.318561  51.945  0.370001
                       5       68.282  0.317111  51.841  0.368701
                       10      68.525  0.351970  52.120  0.406521
                       20      69.091  0.918382  52.796  1.093292
              5        1       68.077  0.222390  51.603  0.256492
                       2       69.553  0.940167  53.341  1.120428
                       5       72.409  1.507859  56.816  1.851842
                       10      74.602  1.452531  59.553  1.806890
                       20      76.732  1.221004  62.293  1.601973
              10       1       68.413  0.277352  51.990  0.321281
                       2       69.357  1.039668  53.117  1.249963
                       5       74.354  0.848045  59.195  1.074946
                       10      77.010  1.021575  62.647  1.353922
                       20      79.972  1.179980  66.674  1.622236
              20       1       68.629  0.992039  52.267  1.189011
                       2       69.580  1.325418  53.399  1.596382
                       5       74.706  1.324698  59.677  1.671604
                       10      78.115  0.785588  64.108  1.058161
                       20      82.651  0.827205  70.455  1.209830
              50       1       68.671  0.822332  52.306  0.972370
                       2       70.030  1.426687  53.936  1.717577
                       5       75.767  1.154035  61.026  1.474678
                       10      79.288  0.584121  65.693  0.808393
                       20      82.750  0.524949  70.582  0.759638
              75       1       68.679  1.130573  52.334  1.362853
                       2       69.557  1.492192  53.385  1.805787
                       5       75.217  1.116798  60.314  1.430397
              100      1       68.801  1.080618  52.471  1.300189
                       2       69.681  1.302201  53.517  1.568738
                       5       75.505  1.257853  60.696  1.608743
                       10      79.017  0.501430  65.317  0.680354
                       20      82.835  0.831503  70.723  1.205687

<h2>Evaluation - Data regime variations</h2>

<p>Evaluate Hebbian models over the datasets, by varying the quantity of training data; only fine-tuned models are considered since during pre-training we can consider the whole dataset.</p>

In [53]:
EXP_ROOT = "./runs"

REGIMES = ['1', '2', '5', '10', '20']

runs = {
    #'UNet': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('unet')),
    #'EM': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('em_unet')),
    #'UAMT': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('uamt_unet')),
    #'CPS': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('cps_unet')),
    'URPC': list(Path(EXP_ROOT + '/GlaS/semi_sup/').glob('urpc_unet')),
}

In [54]:
# Collect predictions scanning runs
predictions = pd.concat([collect_all(k, r, 'test.csv', regimes=REGIMES, ignore_outliers=False) for k, v in runs.items() for r in v], ignore_index=True)

In [55]:
model_grouper = ['model', 'run_number', 'inv_temp', 'regime']
metrics = compute_metrics(predictions, model_grouper, metric_names=['Dice', 'Jaccard'])

display(metrics)

summary = summarize_metrics(metrics, metric_names=['Dice', 'Jaccard'], confidence_level=0.90)

display(summary)

,Model,# Run,Inv Temp,Regime,Dice,Jaccard
0,URPC,0,1,1,68.03,51.55
1,URPC,0,1,2,67.84,51.33
2,URPC,0,1,5,72.85,57.30
3,URPC,0,1,10,76.63,62.11
4,URPC,0,1,20,81.68,69.03
5,URPC,1,1,1,67.85,51.35
6,URPC,1,1,2,68.23,51.78
7,URPC,1,1,5,72.64,57.04
8,URPC,1,1,10,81.41,68.64
9,URPC,1,1,20,85.75,75.05


Dice           Jaccard          
                         Mean   CI 0.9%    Mean   CI 0.9%
Model Inv Temp Regime                                    
URPC  1        1       69.214  0.964375  52.948  1.138827
               2       69.169  1.218885  52.909  1.460061
               5       74.254  1.527695  59.119  1.918430
               10      77.626  1.610150  63.515  2.136674
               20      81.130  2.366412  68.440  3.286478